# PASSO 00 IMPORTS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import requests

# PASSO 01 DATA DESCRIPTION

In [90]:
def get_historical_data(coin, days):
    url = f'https://api.coingecko.com/api/v3/coins/{coin}/market_chart?vs_currency=usd&days={days}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        prices = data['prices']
        df = pd.DataFrame(prices, columns=["Timestamp", "Price (USD)"])
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
        return df
    except Exception as e:
        st.error(f"Erro ao buscar dados históricos: {e}")
        return pd.DataFrame()

In [91]:
historical_data = get_historical_data('bitcoin',days=30)
historical_data.to_parquet('../data/raw/historical_data.parquet')

In [92]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Carregar os dados (supondo que já tenha sido coletado)
df_principal = pd.read_parquet('../data/raw/historical_data.parquet')

# FEATURE ENGINEERING

In [93]:
df = df_principal.copy()
df.tail()

,Timestamp,Price (USD)
714,2025-02-07 18:04:23.151,97416.135935
715,2025-02-07 19:04:05.117,97758.337387
716,2025-02-07 20:04:14.604,96896.289820
717,2025-02-07 21:03:42.904,95746.873954
718,2025-02-07 22:07:44.000,95670.428725


In [94]:
df = df.rename(columns={"Timestamp":"timestamp","Price (USD)":"price (usd)"})
df

,timestamp,price (usd)
0,2025-01-08 22:04:00.926,94477.562204
1,2025-01-08 23:03:48.617,95292.195947
2,2025-01-09 00:03:53.489,95031.897279
3,2025-01-09 01:03:37.169,95105.946616
4,2025-01-09 02:00:33.912,94981.775103
...,...,...
714,2025-02-07 18:04:23.151,97416.135935
715,2025-02-07 19:04:05.117,97758.337387
716,2025-02-07 20:04:14.604,96896.289820
717,2025-02-07 21:03:42.904,95746.873954


In [95]:
df['day_of_week'] = df['timestamp'].dt.dayofweek


# Extrair o número do dia no mês
df['day_of_month'] = df['timestamp'].dt.day

In [96]:
# Média móvel de 7 dias (curto prazo)
df['moving_avg_7'] = df['price (usd)'].rolling(window=7).mean()

# Média móvel de 30 dias (médio prazo)
df['moving_avg_30'] = df['price (usd)'].rolling(window=30).mean()

# Média móvel de 90 dias (longo prazo)
df['moving_avg_90'] = df['price (usd)'].rolling(window=90).mean()

In [97]:
# Calcular a variação percentual diária (retorno)
df['daily_return'] = df['price (usd)'].pct_change()

# Calcular a variação absoluta diária
df['daily_return_abs'] = df['price (usd)'].diff()

In [98]:
df['volatility'] = df['daily_return'].rolling(window=30).std()

In [99]:
# Função para calcular o RSI
def calculate_rsi(data, window):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Calcular o RSI de 14 dias
df['RSI_14'] = calculate_rsi(df['price (usd)'], window=14)

In [100]:
# Média exponencial de 7 dias
df['ema_7'] = df['price (usd)'].ewm(span=7, adjust=False).mean()

# Média exponencial de 30 dias
df['ema_30'] = df['price (usd)'].ewm(span=30, adjust=False).mean()

# Variable Filtering

In [102]:
df = df.dropna()

In [104]:
# Lista das variáveis derivadas
derived_columns = [
    'day_of_week',  'day_of_month', 
    'moving_avg_7', 'moving_avg_30', 'moving_avg_90'
]

# Filtrando apenas as variáveis derivadas
df_derived = df[derived_columns]

# Exibindo as primeiras linhas das variáveis derivadas
print(df_derived.head())

    day_of_week  day_of_month  moving_avg_7  moving_avg_30  moving_avg_90
89            6            12  94255.020324   94409.869845   94120.443525
90            6            12  94382.332934   94432.802425   94126.380649
91            6            12  94537.085908   94448.144608   94122.474207
92            6            12  94700.381243   94465.779783   94123.416097
93            6            12  94802.511388   94477.924626   94121.335772


In [105]:
# Separar variáveis independentes (apenas as derivadas)
X_derived = df_derived

# Variável dependente (preço)
y = df['price (usd)']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_derived, y, test_size=0.3, shuffle=False)

In [106]:
# Inicializar e treinar o modelo de regressão linear
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Prever no conjunto de teste
y_pred = linear_model.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (Random Forest): {mse}')
print(f'Root Mean Squared Error (RMSE) (Random Forest): {mse}')
print(f'Mean Absolute Error (MAE) (Random Forest): {mae}')
print(f'R² (Random Forest): {r2}')

Mean Squared Error (Random Forest): 901801.9403322797
Root Mean Squared Error (RMSE) (Random Forest): 901801.9403322797
Mean Absolute Error (MAE) (Random Forest): 676.2810745502348
R² (Random Forest): 0.8746338266511114


In [107]:
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

# ------------------ Treinamento com XGBoost ------------------
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.4, random_state=42)
xgb_model.fit(X_train, y_train)

# Previsão com XGBoost
xgb_y_pred = xgb_model.predict(X_test)

# Avaliação do XGBoost
xgb_mse = mean_squared_error(y_test, xgb_y_pred)
xgb_rmse = np.sqrt(xgb_mse)
xgb_mae = mean_absolute_error(y_test, xgb_y_pred)
xgb_r2 = r2_score(y_test, xgb_y_pred)

print("------ XGBoost Results ------")
print(f'Mean Squared Error (XGBoost): {xgb_mse}')
print(f'Root Mean Squared Error (RMSE) (XGBoost): {xgb_rmse}')
print(f'Mean Absolute Error (MAE) (XGBoost): {xgb_mae}')
print(f'R² (XGBoost): {xgb_r2}')


# ------------------ Treinamento com Gradient Boosting ------------------
gb_model = GradientBoostingRegressor(n_estimators=10, learning_rate=0.01, random_state=42)
gb_model.fit(X_train, y_train)

# Previsão com Gradient Boosting
gb_y_pred = gb_model.predict(X_test)

# Avaliação do Gradient Boosting
gb_mse = mean_squared_error(y_test, gb_y_pred)
gb_rmse = np.sqrt(gb_mse)
gb_mae = mean_absolute_error(y_test, gb_y_pred)
gb_r2 = r2_score(y_test, gb_y_pred)

print("\n------ Gradient Boosting Results ------")
print(f'Mean Squared Error (Gradient Boosting): {gb_mse}')
print(f'Root Mean Squared Error (RMSE) (Gradient Boosting): {gb_rmse}')
print(f'Mean Absolute Error (MAE) (Gradient Boosting): {gb_mae}')
print(f'R² (Gradient Boosting): {gb_r2}')

------ XGBoost Results ------
Mean Squared Error (XGBoost): 2210466.6534747477
Root Mean Squared Error (RMSE) (XGBoost): 1486.7638189957233
Mean Absolute Error (MAE) (XGBoost): 1214.139675423186
R² (XGBoost): 0.6927066429250026

------ Gradient Boosting Results ------
Mean Squared Error (Gradient Boosting): 13187191.713387758
Root Mean Squared Error (RMSE) (Gradient Boosting): 3631.417314684138
Mean Absolute Error (MAE) (Gradient Boosting): 3084.726078430106
R² (Gradient Boosting): -0.8332492849998132


In [108]:
import joblib

# Caminho local para salvar o modelo
local_model_path = '../models/rl_model.joblib'


# Salvar o modelo
joblib.dump(linear_model, local_model_path)

print(f"Modelo salvo localmente em: {local_model_path}")


Modelo salvo localmente em: ../models/rl_model.joblib


In [109]:
reg_model = joblib.load('../models/rl_model.joblib')


In [110]:
X_train.columns

Index(['day_of_week', 'day_of_month', 'moving_avg_7', 'moving_avg_30',
       'moving_avg_90'],
      dtype='object')

In [111]:
novos_dados = [[5,6,94472.382736,94472.382736,94378.144222]]
predicao = reg_model.predict(novos_dados)

/home/erickfog/ada_tech/Cursos/Produtizacao_de_Modelos_e_MLOPs/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [112]:

print(f'O valor predito é: {predicao[0]:.2f}')

O valor predito é: 94910.63
